# Avaliação Prática – Análise Econômica e Criptofinanceira com Alpha Vantage (Inflação, Câmbio e Bitcoin)

## Pré-Processamento de Dados

### Importação de Bibliotecas

In [108]:
!pip install plotly --upgrade

In [109]:
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler
standardScaler = StandardScaler()
from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import pickle

### Consumo da API Alpha Vantage para Carregamento das Bases de Dados e Conversão para Pandas Dataframe

#### Função "INFLATION" da API para Obteção de Dados sobre a Inflação

In [110]:
url_inflation = 'https://www.alphavantage.co/query?function=INFLATION&apikey=NES1FN2QJITLR295'

In [111]:
r = requests.get(url_inflation)

In [112]:
data = r.json()

In [113]:
inflation_data = data['data']
inflation_df = pd.DataFrame(inflation_data)
inflation_df

,date,value
0,2024-01-01,2.94952520485207
1,2023-01-01,4.11633838374488
2,2022-01-01,8.00279982052121
3,2021-01-01,4.69785886363742
4,2020-01-01,1.23358439630629
...,...,...
60,1964-01-01,1.27891156462591
61,1963-01-01,1.23966942148753
62,1962-01-01,1.19877334820186
63,1961-01-01,1.07072414764724


In [114]:
inflation_df = inflation_df.set_index('date')
inflation_df.index.name = None
inflation_df

,value
2024-01-01,2.94952520485207
2023-01-01,4.11633838374488
2022-01-01,8.00279982052121
2021-01-01,4.69785886363742
2020-01-01,1.23358439630629
...,...
1964-01-01,1.27891156462591
1963-01-01,1.23966942148753
1962-01-01,1.19877334820186
1961-01-01,1.07072414764724


#### Função "FX_DAILY" da API para Obtenção de Dados sobre Câmbio

In [115]:
url_fx_currency = 'https://www.alphavantage.co/query?function=FX_DAILY&outputsize=full&from_symbol=USD&to_symbol=BRL&apikey=NES1FN2QJITLR295'

In [116]:
r = requests.get(url_fx_currency)

In [117]:
data = r.json()

In [118]:
fx_time_series = data['Time Series FX (Daily)']
fx_df = pd.DataFrame.from_dict(fx_time_series, orient='index')
fx_df.index = pd.to_datetime(fx_df.index)
fx_df = fx_df.sort_index()
fx_df

,1. open,2. high,3. low,4. close
2014-11-07,2.55730,2.58100,2.55630,2.55900
2014-11-10,2.56400,2.57150,2.53250,2.55250
2014-11-11,2.56450,2.56600,2.55690,2.56450
2014-11-12,2.56450,2.56670,2.53510,2.56670
2014-11-13,2.56550,2.60420,2.56000,2.59430
...,...,...,...,...
2025-04-21,5.80100,5.80700,5.77830,5.78070
2025-04-22,5.78070,5.80670,5.71750,5.72100
2025-04-23,5.72010,5.72750,5.65680,5.71110
2025-04-24,5.71110,5.71160,5.66090,5.68150


#### Função "DIGITAL_CURRENCY_DAILY" da API para Obtenção de Dados sobre Bitcoin

In [119]:
url_btc = 'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=EUR&apikey=NES1FN2QJITLR295'

In [120]:
r = requests.get(url_btc)

In [121]:
data = r.json()

In [122]:
btc_time_series = data['Time Series (Digital Currency Daily)']
btc_df = pd.DataFrame.from_dict(btc_time_series, orient='index')
btc_df.index = pd.to_datetime(btc_df.index)
btc_df = btc_df.sort_index()
btc_df

,1. open,2. high,3. low,4. close,5. volume
2024-05-11,56524.04000000,57167.19000000,56208.31000000,56496.94000000,198.07642651
2024-05-12,56516.31000000,57435.99000000,56299.00000000,57075.98000000,123.98655238
2024-05-13,57085.97000000,58770.58000000,56418.03000000,58293.80000000,325.85438232
2024-05-14,58298.94000000,58478.73000000,56500.00000000,56924.05000000,320.35856449
2024-05-15,56924.68000000,61022.00000000,56680.09000000,60777.05000000,383.64791289
...,...,...,...,...,...
2025-04-21,74472.62000000,76933.38000000,74357.03000000,75995.58000000,369.45616882
2025-04-22,75997.36000000,82673.74000000,75827.31000000,82333.82000000,550.73983560
2025-04-23,82339.52000000,83264.20000000,80800.00000000,82718.26000000,490.53056792
2025-04-24,82741.42000000,82860.82000000,80606.52000000,82682.47000000,227.68974180


#### Visualização dos Dataframes

In [123]:
inflation_df

,value
2024-01-01,2.94952520485207
2023-01-01,4.11633838374488
2022-01-01,8.00279982052121
2021-01-01,4.69785886363742
2020-01-01,1.23358439630629
...,...
1964-01-01,1.27891156462591
1963-01-01,1.23966942148753
1962-01-01,1.19877334820186
1961-01-01,1.07072414764724


In [124]:
fx_df

,1. open,2. high,3. low,4. close
2014-11-07,2.55730,2.58100,2.55630,2.55900
2014-11-10,2.56400,2.57150,2.53250,2.55250
2014-11-11,2.56450,2.56600,2.55690,2.56450
2014-11-12,2.56450,2.56670,2.53510,2.56670
2014-11-13,2.56550,2.60420,2.56000,2.59430
...,...,...,...,...
2025-04-21,5.80100,5.80700,5.77830,5.78070
2025-04-22,5.78070,5.80670,5.71750,5.72100
2025-04-23,5.72010,5.72750,5.65680,5.71110
2025-04-24,5.71110,5.71160,5.66090,5.68150


In [125]:
btc_df

,1. open,2. high,3. low,4. close,5. volume
2024-05-11,56524.04000000,57167.19000000,56208.31000000,56496.94000000,198.07642651
2024-05-12,56516.31000000,57435.99000000,56299.00000000,57075.98000000,123.98655238
2024-05-13,57085.97000000,58770.58000000,56418.03000000,58293.80000000,325.85438232
2024-05-14,58298.94000000,58478.73000000,56500.00000000,56924.05000000,320.35856449
2024-05-15,56924.68000000,61022.00000000,56680.09000000,60777.05000000,383.64791289
...,...,...,...,...,...
2025-04-21,74472.62000000,76933.38000000,74357.03000000,75995.58000000,369.45616882
2025-04-22,75997.36000000,82673.74000000,75827.31000000,82333.82000000,550.73983560
2025-04-23,82339.52000000,83264.20000000,80800.00000000,82718.26000000,490.53056792
2025-04-24,82741.42000000,82860.82000000,80606.52000000,82682.47000000,227.68974180


#### Unificação dos Dataframes

In [126]:
def last_year_inflatio(df):
    df.index = pd.to_datetime(df.index)
    ultimo_ano = df.index.year.max()
    df_ultimo_ano = df[df.index.year == ultimo_ano]

    return df_ultimo_ano

last_year_inflation_df = last_year_inflatio(inflation_df)
last_year_inflation_df

,value
2024-01-01,2.94952520485207


In [127]:
last_year_inflation_df = last_year_inflation_df.astype(float)

In [128]:
last_year_inflation_df.describe()

,value
count,1.000000
mean,2.949525
std,NaN
min,2.949525
25%,2.949525
50%,2.949525
75%,2.949525
max,2.949525


In [129]:
last_year_inflation_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1 entries, 2024-01-01 to 2024-01-01
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   value   1 non-null      float64
dtypes: float64(1)
memory usage: 16.0 bytes


In [130]:
def last_ninety_days_dfs(df):
    df.index = pd.to_datetime(df.index)
    end_date = pd.Timestamp("2024-12-31")
    start_date = end_date - pd.Timedelta(days=89)
    last_ninety_days_df = df[(df.index >= start_date) & (df.index <= end_date)]

    return last_ninety_days_df

In [131]:
last_ninety_days_fx_df = last_ninety_days_dfs(fx_df)
last_ninety_days_fx_df

,1. open,2. high,3. low,4. close
2024-10-03,5.44410,5.51080,5.43790,5.46610
2024-10-04,5.46610,5.51860,5.45110,5.45110
2024-10-07,5.45510,5.49750,5.41960,5.49700
2024-10-08,5.48790,5.53980,5.48000,5.52840
2024-10-09,5.52840,5.59970,5.51780,5.59420
...,...,...,...,...
2024-12-25,6.19060,6.19060,6.19060,6.19060
2024-12-26,6.17380,6.19770,6.14580,6.18280
2024-12-27,6.17110,6.21480,6.16890,6.20110
2024-12-30,6.20110,6.24200,6.15120,6.18400


In [132]:
last_ninety_days_fx_df = last_ninety_days_fx_df.astype(float)

In [133]:
last_ninety_days_fx_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 64 entries, 2024-10-03 to 2024-12-31
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   1. open   64 non-null     float64
 1   2. high   64 non-null     float64
 2   3. low    64 non-null     float64
 3   4. close  64 non-null     float64
dtypes: float64(4)
memory usage: 2.5 KB


In [134]:
last_ninety_days_fx_df.describe()

,1. open,2. high,3. low,4. close
count,64.000000,64.000000,64.000000,64.000000
mean,5.844484,5.892328,5.810652,5.856353
std,0.216935,0.219076,0.207347,0.216523
min,5.444100,5.497500,5.419600,5.451100
25%,5.690725,5.722425,5.670900,5.699750
50%,5.787350,5.824600,5.757350,5.792900
75%,6.046250,6.090050,6.008525,6.050675
max,6.289600,6.313900,6.190600,6.289600


In [135]:
last_ninety_days_btc_df = last_ninety_days_dfs(btc_df)
last_ninety_days_btc_df

,1. open,2. high,3. low,4. close,5. volume
2024-10-03,54893.39000000,55719.54000000,54237.15000000,55053.60000000,325.96403893
2024-10-04,55047.76000000,56945.23000000,54785.63000000,56572.45000000,229.25658222
2024-10-05,56588.84000000,56831.74000000,56187.41000000,56505.53000000,58.20658723
2024-10-06,56522.61000000,57354.13000000,56300.00000000,57248.79000000,92.42354624
2024-10-07,57256.48000000,58723.00000000,56600.00000000,56722.36000000,207.05337233
...,...,...,...,...,...
2024-12-27,91774.04000000,93579.09000000,89634.61000000,90412.16000000,373.71835226
2024-12-28,90390.29000000,91634.81000000,90225.50000000,91231.22000000,132.80990947
2024-12-29,91235.75000000,91271.67000000,89000.00000000,89702.09000000,250.31280693
2024-12-30,89692.71000000,91260.41000000,87834.22000000,88994.94000000,447.55649032


In [136]:
last_ninety_days_btc_df = last_ninety_days_btc_df.astype(float)

In [137]:
last_ninety_days_btc_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 90 entries, 2024-10-03 to 2024-12-31
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   1. open    90 non-null     float64
 1   2. high    90 non-null     float64
 2   3. low     90 non-null     float64
 3   4. close   90 non-null     float64
 4   5. volume  90 non-null     float64
dtypes: float64(5)
memory usage: 4.2 KB


In [138]:
last_ninety_days_btc_df.describe()

,1. open,2. high,3. low,4. close,5. volume
count,90.000000,90.000000,90.000000,90.000000,90.000000
mean,78584.094444,80355.126111,77167.897111,78975.174444,456.248073
std,15456.969021,15805.888888,14913.375057,15299.228074,362.698113
min,54893.390000,55719.540000,53904.090000,55053.600000,48.986074
25%,62862.537500,63625.332500,61964.112500,62940.360000,220.175993
50%,85347.655000,87337.915000,84254.055000,85544.665000,349.702416
75%,92137.102500,94266.650000,90171.887500,92147.542500,549.567235
max,101163.080000,103000.000000,100301.000000,101171.900000,1838.122623


In [140]:
inflation_value = last_year_inflation_df['value'].values[0]

fx_open_mean = last_ninety_days_fx_df['1. open'].mean()
fx_high_mean = last_ninety_days_fx_df['2. high'].mean()
fx_low_mean = last_ninety_days_fx_df['3. low'].mean()
fx_close_mean = last_ninety_days_fx_df['4. close'].mean()

btc_open_mean = last_ninety_days_btc_df['1. open'].mean()
btc_high_mean = last_ninety_days_btc_df['2. high'].mean()
btc_low_mean = last_ninety_days_btc_df['3. low'].mean()
btc_close_mean = last_ninety_days_btc_df['4. close'].mean()
btc_volume_mean = last_ninety_days_btc_df['5. volume'].mean()

cripto_analisys_df = pd.DataFrame({
    'inflation_mean': inflation_value,
    'fx_open_mean': fx_open_mean,
    'fx_high_mean': fx_high_mean,
    'fx_low_mean': fx_low_mean,
    'fx_close_mean': fx_close_mean,
    'btc_open_mean': btc_open_mean,
    'btc_high_mean': btc_high_mean,
    'btc_low_mean': btc_low_mean,
    'btc_close_mean': btc_close_mean,
    'btc_volume_mean': btc_volume_mean,
}, index=['2024'])
cripto_analisys_df

,inflation_mean,fx_open_mean,fx_high_mean,fx_low_mean,fx_close_mean,btc_open_mean,btc_high_mean,btc_low_mean,btc_close_mean,btc_volume_mean
2024,2.949525,5.844484,5.892328,5.810652,5.856353,78584.094444,80355.126111,77167.897111,78975.174444,456.248073
